In [20]:
# Imports
import requests
import getpass

In [21]:
base_url = 'http://localhost:8080'

In [22]:
# Users Endpoints
users_url = f'{base_url}/users'

# Create account
def create_account(email, password, username):
    body = {
        'email': email,
        'pass': password,
        'username': username
    }
    r = requests.post(users_url, json=body)
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

# Login to existing account
def login(email, password):
    body = {
        'email': email,
        'pass': password
    }
    r = requests.post(f'{users_url}/login', json=body)
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

# Delete existing account
def delete_account(user_id):
    r = requests.delete(f'{users_url}/{user_id}')
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

In [23]:
# Recipes Endpoints
recipes_url = f'{base_url}/recipes'

# Create recipe
def create_recipe(recipe_body):
    r = requests.post(recipes_url, json=recipe_body)
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

# Update recipe
def update_recipe(recipe_id, recipe_body):
    r = requests.put(f'{recipes_url}/{recipe_id}', json=recipe_body)
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

# Get Recipe by criteria
def get_recipe_by_criteria(title=None, user=None, tags=[]):
    filters_arr = []
    if title:
        filters_arr.append(f'title={title}')
    if user != None:
        filters_arr.append(f'user={user}')
    if tags:
        filters_arr.append(f'tags={tags_str}')
    filters = '&'.join(filters_arr)
    if len(filters_arr) > 0:
        filters = '?' + filters
    r = requests.get(f'{recipes_url}{filters}')
    
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

def get_recipe_by_id(recipe_id):
    r = requests.get(f'{recipes_url}/{recipe_id}')
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

# Get tools
def get_tools():
    r = requests.get(f'{recipes_url}/tools')
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

# Get tags
def get_tags():
    r = requests.get(f'{recipes_url}/tags')
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

# Delete recipe by ID
def delete_recipe(recipe_id):
    r = requests.delete(f'{recipes_url}/{recipe_id}')
    return {
        'status_code': r.status_code,
        'data': r.json()
    }

In [5]:
# Auth Flow
def auth_flow():
    auth_method = None
    while auth_method is None or auth_method not in ['l', 's']:
        if auth_method is not None:
            print('Inalid input. Please enter "l" to log in or "s" to sign up.\n')
        auth_method = input('Do you want to login (enter: "l") or sign up (enter "s")? ')

    print()
    user_id = None

    if auth_method == 's':
        while user_id is None:
            email = input('Enter email: ')
            password = getpass.getpass('Enter password: ')
            username = input('Enter username: ')
            create_res = create_account(email, password, username)
            if create_res['status_code'] != 201:
                print(f'Error: {create_res["data"]["message"]}. Please try again.\n')
            else:
                user_id = create_res['data']['userID']

    if auth_method == 'l':
        while user_id is None:
            email = input('Enter email: ')
            password = getpass.getpass('Enter password: ')
            login_res = login(email, password)
            if login_res['status_code'] != 200:
                print(f'Error: {login_res["data"]["message"]}. Please try again.\n')
            elif not login_res['data']['authenticated']:
                print(f'Error: the email or password is incorrect. Please try again.\n')
            else:
                user_id = login_res['data']['userID']
                
    return user_id

In [24]:
def list_tools():
    tools_resp = get_tools()
    if tools_resp['status_code'] == 200:
        print('Available tools:')
        for i, item in enumerate(tools_resp['data']['tools']):
            print(f'{i+1})', item['toolName'])
        return True
    else:
        print('Error retrieving tools. Please try restarting the application.')
        return False
            
def list_tags():
    tags_resp = get_tags()
    if tags_resp['status_code'] == 200:
        print('Available tags:')
        for i, item in enumerate(tags_resp['data']['tags']):
            print(f'{i+1})', item['tagName'])
        return True
    else:
        print('Error retrieving tags. Please try restarting the application.')
        return False

In [25]:
def enter_recipe_details(author_id):
    try:
        title = input('Enter title of recipe: ')
        assert(len(title) > 0)
        
        num_ingredients = int(input('How many ingredients required (1-10): '))
        assert(1 <= num_ingredients <= 10)
        ingredients = []
        for i in range(num_ingredients):
            name = input('Enter description of ingredient (cannot contain @@ or ##): ')
            assert(len(name) > 0)
            assert(not ('@@' in name or '##' in name))
            amount = int(input('Enter amount of ingredient used in recipe (1+): '))
            ingredient = {
                'ingredientName': name,
                'amount': amount
            }
            ingredients.append(ingredient)
        
        prep_time = int(input('Enter time it takes to prepare recipe in minutes: '))
        assert(prep_time > 0)
        
        servings = int(input('Enter number of servings (1+): '))
        assert(servings > 0)
        
        instructions = input('Enter the recipe instructions (250 words or less): ')
        assert(len(instructions.split(' ')) <= 250)
        
        tools_success = list_tools()
        if not tools_success:
            raise ValueError()
        tools_list = input('Enter required tools for recipe separated with commas (e.g. "1,3,10"): ')
        tools = [int(t) for t in tools_list.split(',')]
        
        tags_success = list_tags()
        if not tags_success:
            raise ValueError()
        tags_list = input('Enter tags that match the recipe separated with commas (e.g. "1,3,10"): ')
        tags = [int(t) for t in tags_list.split(',')]
    except:
        print('Invalid input received. Please follow the prompts and enter the recipe again.\n')
        enter_recipe_details(author_id)
    
    return {
        'authorID': author_id,
        'title': title,
        'prepTime': prep_time,
        'servings': servings,
        'instructions': instructions,
        'ingredients': ingredients,
        'tools': tools,
        'tags': tags
    }

In [27]:
# Main

user_id = -1 #auth_flow()
while True:
    action_menu = '''
1) Create a new recipe
2) Update one of your existing recipes
3) Search for a recipe
4) Delete one of your existing recipes
5) Switch accounts
6) Delete your account
7) End the program
Enter a number to perform the corresponding action: '''
    action_id = input(action_menu)
    print(action_id)
    if action_id == '1':
        # take recipe details input
        # create recipe
        body = enter_recipe_details(user_id)
        create_resp = create_recipe(body)
        if create_resp['status_code'] == 201:
            print(f'Successfully created recipe with ID {create_resp["data"]["recipeID"]}')
        else:
            print(f'Error: {create_resp["data"]["message"]}. Please try again.')
    elif action_id == '2':
        # list existing user recipes (get_recipes)
        # take input for recipe_id
        # take input for recipe details
        # update recipe
        user_recipes = get_recipes_by_criteria('', user_id)
    elif action_id == '3':
        print('TODO')
        # take input for recipe name
        # take input for list of tags
        # call get recipes
    elif action_id == '4':
        print('TODO')
        # list recipes
        # take input for recipe_id
        # delete recipe
    elif action_id == '5':
        user_id = auth_flow()
    elif action_id == '6':
        delete_account(user_id)
        user_id = auth_flow()
    elif action_id == '7':
        break
    else:
        print('Invalid input. Please enter a valid number from 1-7.')

print('Thank you for using SimpleRecipes')
# exit()


1) Create a new recipe
2) Update one of your existing recipes
3) Search for a recipe
4) Delete one of your existing recipes
5) Switch accounts
6) Delete your account
7) End the program
Enter a number to perform the corresponding action: 1
1
Enter title of recipe: title
How many ingredients required (1-10): 1
Enter description of ingredient (cannot contain @@ or ##): ingredient 1
Enter amount of ingredient used in recipe (1+): 11
Enter time it takes to prepare recipe in minutes: 12
Enter number of servings (1+): 2
Enter the recipe instructions (250 words or less): some instructions
Available tools:
1) Skillet
2) Frying Pan
3) Saucepot
4) Soup Pot
5) Wok
6) Baking Sheet
7) Baking Dish
8) Tongs
9) Spatula
10) Whisk
11) Spoon
12) Strainer
13) Chopsticks
14) Fork
15) Measuring Cup Set
16) Measuring Spoon Set
17) Chef's Knife
18) Paring Knife
19) Scissors
20) Vegetable Peeler
21) Grater
22) Rolling Pin
23) Masher
24) Butter Knife
Enter required tools for recipe separated with commas (e.g. "

KeyboardInterrupt: Interrupted by user